In [ ]:
import refinitiv.data as rd
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cufflinks as cf
# enable multiple outputs from each Jupyter block
cf.go_offline()

rd.open_session()


In [5]:
def get_data(ric_chain, show_chart=True):
    # retrieving target Chain RIC and assigning to df yc
    yc = rd.get_data(rd.discovery.Chain(ric_chain), ['DSPLY_NAME'])

    # creating array of RICs from the chain RIC in df yc (droping first 3 rows since they are invalid RICs)
    riclist = yc.drop(yc.index[0:5])['Instrument'].tolist()

    # retrieves time series for riclist for last 9 months
    end_date = datetime.now()
    start_date = end_date - timedelta(days=270)
    ust_yc2 = rd.get_history(riclist, fields="B_YLD_1", start=start_date,
                             end=end_date, interval="daily")

    # creating location index in df ust_yc2 for a specific point in time.
    total = int(len(ust_yc2))
    total30 = int(len(ust_yc2)/12)
    total90 = int(len(ust_yc2)/4)
    total180 = int(len(ust_yc2)/2)
    total270 = int(len(ust_yc2)/12)*9

    # calculating the historical starting point for each time period within dataframe ust_yc2 (starting from the tail)
    # output to be used
    pctchg1m = ((ust_yc2.iloc[-1] / ust_yc2.iloc[total - total30]) - 1)
    pctchg3m = ((ust_yc2.iloc[-1] / ust_yc2.iloc[total - total90]) - 1)
    pctchg6m = ((ust_yc2.iloc[-1] / ust_yc2.iloc[total - total180]) - 1)
    pctchg9m = ((ust_yc2.iloc[-1] / ust_yc2.iloc[total - total270]) - 1)
    pctchg1y = ((ust_yc2.iloc[-1] / ust_yc2.iloc[0]) - 1)

    # creating new dataframe from each individual pctchg series
    frame = {'1M': pctchg1m, '3M': pctchg3m, '6M': pctchg6m, '9M': pctchg9m}
    pct_chg_matrix = pd.DataFrame(frame)

    if show_chart:
        # chart curve and plot % change over terms
        ycs = pct_chg_matrix.iplot(kind='bar', yTitle='% Chg', title='Yield Curve Shifts', theme='solar', colors=[
                                   '#6978F7', '#A325E9', '#96E05D', '#4A7FB9', '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'])
    return pct_chg_matrix


In [ ]:
rd.close_session()